In [2]:
import pickle
import numpy as np


X = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['x']
print("shape of X:", np.shape(X))

y = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['y']
print("shape of y:", np.shape(y))

def functionXY(Xinput):
  for i, input in enumerate(X):
    if np.array_equal(Xinput, input):
      return y[i]
  return None

print(X[56])
print(y[56])

print(functionXY(X[56]))

shape of X: (98694, 7)
shape of y: (98694,)
[0.05436893 0.54681128 0.41890404 0.90909091 0.0797649  0.2263727
 0.1648951 ]
10.44
10.44


In [4]:
import numpy as np
import random

def objective_function(x):
    # x is a tuple of discrete feature values, corresponding to each feature in X

    # Convert the input tuple to a NumPy array
    x = np.array(x)

    # Find the closest data point in the dataset
    distances = np.sum((X - x)**2, axis=1)
    closest_index = np.argmin(distances)

    # Snap the input to the closest data point
    snapped_input = X[closest_index]

    return y[closest_index]

def clip_position(x):
    x = np.array(x)
    
    distances = np.sum((X - x)**2, axis=1)
    closest_index = np.argmin(distances)
    
    snapped_input = X[closest_index]
    
    return snapped_input

class Particle:
    def __init__(self, num_dimensions, lower_bound, upper_bound, init_position, index):
        self.position = init_position
        self.velocity = np.zeros(num_dimensions)
        self.best_position = np.copy(self.position)
        self.best_fitness = objective_function(self.position)
        self.index = index

class PSO:
    def __init__(self, num_particles, num_dimensions, lower_bound, upper_bound, w, c1, c2, num_iterations, init_positions):
        self.num_particles = num_particles
        self.num_dimensions = num_dimensions
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.num_iterations = num_iterations
        self.particles = [Particle(num_dimensions, lower_bound, upper_bound, init_positions[ind], ind) for ind in range(num_particles)]
        self.global_best_position = np.copy(self.particles[0].position)
        self.global_best_fitness = self.particles[0].best_fitness

    def update_particles(self):
        for particle in self.particles:
            # Update velocities
            particle.velocity = (self.w * particle.velocity
                                 + self.c1 * np.random.rand() * (particle.best_position - particle.position)
                                 + self.c2 * np.random.rand() * (self.global_best_position - particle.position))

            # Update positions
            particle.position = particle.position + particle.velocity

            # Clip positions to stay within bounds
            particle.position = np.clip(particle.position, self.lower_bound, self.upper_bound)
            particle.position = clip_position(particle.position)
            
    def update_personal_and_global_bests(self):
        for particle in self.particles:
            # Evaluate fitness
            current_fitness = objective_function(particle.position)

            # Update personal best
            if current_fitness > particle.best_fitness:
                particle.best_position = np.copy(particle.position)
                particle.best_fitness = current_fitness

                # Update global best
                if current_fitness > self.global_best_fitness:
                    self.global_best_position = np.copy(particle.position)
                    self.global_best_fitness = current_fitness
            
            #print("Particle #", particle.index, ": ")
            #print("-> Current Position: ", particle.position)
            #print("-> Current Velocity: ", particle.velocity)
            #print("-> Current Output: ", functionXY(particle.position))
            #print("-> Personal Best Position: ",  particle.best_position)
            
        #print("---> Global Best Position: ", self.global_best_position, " <---")
        #print("---> Global Best Output: ", functionXY(self.global_best_position), " <---")    
        
    def eliminate(self):
        #print(self.particles[1].position)
        #print(functionXY(self.particles[1].position))
        #print(clip_position(self.particles[1].position))
        #print(functionXY(clip_position(self.particles[1].position)))
        #print( (self.particles[0].position in X) )
        #print( (self.particles[1].position in X) )
        #print( (clip_position(self.particles[1].position) in X))
        
        num_to_eliminate = int((self.num_particles / (self.num_iterations - 1)))
        if not self.particles:
            return []
        
        for _ in range(num_to_eliminate):
            min_index = 0
            for j in range(1, len(self.particles)):
                
                #print( type(functionXY(self.particles[j].position)) )
                #print( type(functionXY(self.particles[min_index].position)) )
                      
                if (functionXY(clip_position(self.particles[j].position)) < functionXY(clip_position(self.particles[min_index].position))):
                    min_index = j
            
            
            self.particles.pop(min_index)
            print("eliminated index ", min_index)
        
    def optimize(self):
        for i in range(self.num_iterations):
            print("Iteration #", i, ": ")
            print("x-x-x-x-x-x-x-x-x-x-x")
            self.update_particles()
            self.update_personal_and_global_bests()
            if (i != (self.num_iterations - 1)):
                self.eliminate()
            print("Particles remaining after Iteration #", i, ": ", len(self.particles))
            

        return self.global_best_position, self.global_best_fitness

# Example usage
num_particles = 10
num_dimensions = 7
lower_bound = 0
upper_bound = 1.0
w = 0.8
c1 = 0.8
c2 = 0.8
num_iterations = 10

initial_positions_PSO = random.choices(X, k=num_particles)
print(initial_positions_PSO)


pso = PSO(num_particles, num_dimensions, lower_bound, upper_bound, w, c1, c2, num_iterations, initial_positions_PSO)
best_position, best_fitness = pso.optimize()

print("Best position found: ", best_position)
print("Best fitness found: ", best_fitness)


[array([0.07184466, 0.59287897, 0.58147894, 0.82828283, 0.05737476,
       0.12961663, 0.1627972 ]), array([0.11067961, 0.3872041 , 0.56486826, 0.77777778, 0.03582424,
       0.13144221, 0.10699301]), array([0.24466019, 0.04995385, 0.15761623, 0.47474747, 0.01035544,
       0.06417638, 0.06475524]), array([0.13592233, 0.26579897, 0.47119664, 0.72727273, 0.02742793,
       0.11136076, 0.1       ]), array([0.10291262, 0.36957436, 0.50625529, 0.75757576, 0.03722362,
       0.13228479, 0.10839161]), array([0.06990291, 0.50473641, 0.48294126, 0.83838384, 0.05933389,
       0.17188597, 0.15230769]), array([0.10485437, 0.30909538, 0.42664868, 0.74747475, 0.03666387,
       0.17216683, 0.13160839]), array([0.15533981, 0.21356615, 0.43183764, 0.67676768, 0.02267003,
       0.12329729, 0.09384615]), array([0.08349515, 0.43270051, 0.48179734, 0.80808081, 0.04925833,
       0.17778402, 0.13944056]), array([0.14951456, 0.17050359, 0.33066418, 0.65656566, 0.0229499 ,
       0.06403595, 0.05426573])]